<a href="https://colab.research.google.com/github/alikaiser12/AI/blob/main/PNG_renamed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 0: Install dependencies
!pip install pytesseract pillow google-colab --quiet
!apt-get install tesseract-ocr -y

from PIL import Image
import pytesseract
import os
import shutil
from google.colab import files
from zipfile import ZipFile
import hashlib
import re

# -------------------------
# Step 1: Set folders
input_folder = "/content/drive/MyDrive/YC/cactus_ai"  # Upload your screenshots here
output_folder = "/content/renamed_screenshots"
os.makedirs(output_folder, exist_ok=True)

# -------------------------
# Step 2: Function to remove unwanted characters & make a clean name
def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r"[^a-z0-9]+", "_", text)  # only letters/numbers/underscores
    text = re.sub(r"_+", "_", text)  # collapse multiple underscores
    return text.strip("_")[:30]  # limit length

# -------------------------
# Step 3: Extract text from image (OCR)
def extract_text(img_path):
    try:
        img = Image.open(img_path)
        text = pytesseract.image_to_string(img)
        if text.strip() == "":
            return "unknown"
        return clean_text(text)
    except:
        return "error"

# -------------------------
# Step 4: Compute image hash (for duplicate detection)
def image_hash(img_path):
    img = Image.open(img_path).convert("RGB")
    return hashlib.md5(img.tobytes()).hexdigest()

# -------------------------
# Step 5: Process images
seen_hashes = set()
name_counter = {}  # To handle multiple images with same content-based name

for filename in sorted(os.listdir(input_folder)):
    if not filename.lower().endswith(".png"):
        continue

    old_path = os.path.join(input_folder, filename)
    img_hash = image_hash(old_path)

    if img_hash in seen_hashes:
        print(f"Skipping duplicate: {filename}")
        continue
    seen_hashes.add(img_hash)

    # Generate human-readable name
    base_name = extract_text(old_path)

    # If multiple images have same base_name, add counter
    if base_name in name_counter:
        name_counter[base_name] += 1
    else:
        name_counter[base_name] = 1

    new_name = f"{base_name}_{name_counter[base_name]}.png"
    new_path = os.path.join(output_folder, new_name)

    shutil.copy(old_path, new_path)
    print(f"Saved: {new_name}")

# -------------------------
# Step 6: Zip renamed screenshots
zip_name = "/content/renamed_screenshots.zip"
with ZipFile(zip_name, 'w') as zipf:
    for root, dirs, files_in in os.walk(output_folder):
        for file_in in files_in:
            zipf.write(os.path.join(root, file_in), arcname=file_in)

# -------------------------
# Step 7: Download
files.download(zip_name)
print("✅ All screenshots renamed clearly, duplicates removed, and zipped!")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Saved: do_papepateon_no_microphone_ee_1.png
Saved: ppd_dat_oume_x_gap_pe_ea_eer_r_1.png
Saved: le_ll_ae_ea_se_dae_r_wa_ll_a_a_1.png
Saved: per_dat_crime_x_ts_oo_er_enn_i_1.png
Saved: ce_se_ran_renting_ape_research_1.png
Saved: pep_ort_ontine_eae_ce_messing__1.png
Saved: per_dat_crime_x_ts_ra_o_pry_i__1.png
Saved: pep_ort_ontine_eae_ce_en_ene_s_1.png
Saved: or_i_c_canealscuton_gp_nctiiog_1.png
Saved: le_ll_ae_ea_se_dae_r_wa_a_al_r_1.png
Saved: bmages_in_cardial_eeous_rors_a_1.png
Saved: v_paperpal_unlock_the_power_of_1.png
Saved: per_dat_crime_x_ere_oo_er_rn_i_1.png
Saved: ppd_dat_oume_x_gap_pe_a_ee_e_i_1.png
Saved: prod_dat_crime_ee_baa_nnsuaing_1.png
Saved: bimages_in_cardial_fer_ono_pe__1.png
Saved: prod_dat_crime_ee_baa_en_en_z__1.png
Saved: pep_ort_ontine_eae_ce

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ All screenshots renamed clearly, duplicates removed, and zipped!


In [ ]:
# ===============================
# Step 0: Install dependencies
# ===============================
!pip install pillow pytesseract google-colab --quiet
!apt-get install tesseract-ocr -y

from PIL import Image
import pytesseract
import os
import shutil
from google.colab import files
from zipfile import ZipFile
import hashlib

# ===============================
# Step 1: Set folders
# ===============================
input_folder = "/content/drive/MyDrive/YC/cactus_ai"  # Upload your screenshots here
output_folder = "/content/renamed_screenshots"
os.makedirs(output_folder, exist_ok=True)

# ===============================
# Step 2: Compute image hash for duplicate detection
# ===============================
def image_hash(img_path):
    img = Image.open(img_path).convert("RGB")
    return hashlib.md5(img.tobytes()).hexdigest()

# ===============================
# Step 3: Function to extract OCR text from image
# (we use this only to decide type of page, but not raw text)
# ===============================
def extract_page_type(img_path):
    try:
        img = Image.open(img_path)
        text = pytesseract.image_to_string(img).lower()
        # Very simple rules to categorize
        if any(word in text for word in ["login", "sign in"]):
            return "login_page"
        elif any(word in text for word in ["signup", "register", "create account"]):
            return "signup_page"
        elif any(word in text for word in ["dashboard", "overview"]):
            return "dashboard"
        elif any(word in text for word in ["landing", "home"]):
            return "landing_page"
        elif any(word in text for word in ["settings", "preferences"]):
            return "settings_page"
        else:
            return "other_page"
    except:
        return "unknown_page"

# ===============================
# Step 4: Process images
# ===============================
seen_hashes = set()
name_counter = {}  # To handle multiple images of same type

for filename in sorted(os.listdir(input_folder)):
    if not filename.lower().endswith(".png"):
        continue

    old_path = os.path.join(input_folder, filename)
    img_hash = image_hash(old_path)

    if img_hash in seen_hashes:
        print(f"Skipping duplicate: {filename}")
        continue
    seen_hashes.add(img_hash)

    # Determine page type
    page_type = extract_page_type(old_path)

    # Counter for same page type
    if page_type in name_counter:
        name_counter[page_type] += 1
    else:
        name_counter[page_type] = 1

    new_name = f"{page_type}_{name_counter[page_type]}.png"
    new_path = os.path.join(output_folder, new_name)

    shutil.copy(old_path, new_path)
    print(f"Saved: {new_name}")

# ===============================
# Step 5: Zip renamed screenshots
# ===============================
zip_name = "/content/renamed_screenshots.zip"
with ZipFile(zip_name, 'w') as zipf:
    for root, dirs, files_in in os.walk(output_folder):
        for file_in in files_in:
            zipf.write(os.path.join(root, file_in), arcname=file_in)

# ===============================
# Step 6: Download
# ===============================
files.download(zip_name)
print("✅ All screenshots renamed clearly, duplicates removed, zipped for download!")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Saved: other_page_1.png
Saved: other_page_2.png
Saved: other_page_3.png
Saved: other_page_4.png
Saved: other_page_5.png
Saved: other_page_6.png
Saved: other_page_7.png
Saved: other_page_8.png
Saved: other_page_9.png
Saved: other_page_10.png
Saved: other_page_11.png
Saved: other_page_12.png
Saved: other_page_13.png
Saved: other_page_14.png
Saved: other_page_15.png
Saved: other_page_16.png
Saved: other_page_17.png
Saved: other_page_18.png
Saved: other_page_19.png
Saved: other_page_20.png
Saved: other_page_21.png
Saved: other_page_22.png
Saved: other_page_23.png
Saved: other_page_24.png
Saved: other_page_25.png
Saved: dashboard_1.png
Saved: other_page_26.png
Saved: other_page_27.png
Saved: other_page_28.png
Saved: other_page_29.png
Saved: other_page_30.png
Saved: other

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ All screenshots renamed clearly, duplicates removed, zipped for download!


In [ ]:
# Step 0: Install dependencies
!pip install transformers torch torchvision pillow google-colab --quiet

from PIL import Image
import os
import shutil
from google.colab import files
from zipfile import ZipFile
import hashlib
import re

from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# Step 1: Set folders
input_folder = "/content/drive/MyDrive/YC/cactus_ai"  # Upload your screenshots here
output_folder = "/content/renamed_screenshots"
os.makedirs(output_folder, exist_ok=True)

# Step 2: Load BLIP model for image captioning
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# Step 3: Clean text for filenames
def clean_text(text):
    text = text.lower().strip()
    text = re.sub(r"[^a-z0-9]+", "_", text)
    text = re.sub(r"_+", "_", text)
    return text.strip("_")[:30]

# Step 4: Compute image hash for duplicate detection
def image_hash(img_path):
    img = Image.open(img_path).convert("RGB")
    return hashlib.md5(img.tobytes()).hexdigest()

# Step 5: Map caption to clear page type
def map_caption_to_type(caption):
    caption = caption.lower()
    if any(word in caption for word in ["login", "sign in"]):
        return "login_page"
    elif any(word in caption for word in ["signup", "register", "create account"]):
        return "signup_page"
    elif any(word in caption for word in ["dashboard", "overview"]):
        return "dashboard"
    elif any(word in caption for word in ["landing", "home page", "homepage"]):
        return "landing_page"
    elif any(word in caption for word in ["settings", "preferences"]):
        return "settings_page"
    else:
        return "other_page"

# Step 6: Process images
seen_hashes = set()
name_counter = {}

for filename in sorted(os.listdir(input_folder)):
    if not filename.lower().endswith(".png"):
        continue

    old_path = os.path.join(input_folder, filename)
    img_hash = image_hash(old_path)

    if img_hash in seen_hashes:
        print(f"Skipping duplicate: {filename}")
        continue
    seen_hashes.add(img_hash)

    # Load image and generate caption
    raw_image = Image.open(old_path).convert("RGB")
    inputs = processor(raw_image, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)

    # Map to clear page type
    page_type = map_caption_to_type(caption)

    # Counter for same page type
    if page_type in name_counter:
        name_counter[page_type] += 1
    else:
        name_counter[page_type] = 1

    new_name = f"{page_type}_{name_counter[page_type]}.png"
    new_path = os.path.join(output_folder, new_name)

    shutil.copy(old_path, new_path)
    print(f"Saved: {new_name} ({caption})")

# Step 7: Zip renamed screenshots
zip_name = "/content/renamed_screenshots.zip"
with ZipFile(zip_name, 'w') as zipf:
    for root, dirs, files_in in os.walk(output_folder):
        for file_in in files_in:
            zipf.write(os.path.join(root, file_in), arcname=file_in)

# Step 8: Download
files.download(zip_name)
print("✅ All screenshots renamed clearly, duplicates removed, zipped for download!")


Saved: other_page_1.png (the website for the new website for the new website)
Saved: other_page_2.png (the new version of the chrome browser)
Saved: other_page_3.png (the screen of the new google search page)
Saved: other_page_4.png (the screen shot of the twitter page)
Saved: other_page_5.png (the new version of the new version of the chrome browser)
Saved: other_page_6.png (the screen shot of the new google search page)
Saved: other_page_7.png (the new version of the new version of the chrome browser)
Saved: other_page_8.png (the new version of the new version of the new version of the android)
Saved: other_page_9.png (the wordpress page in the wordpress editor)
Saved: other_page_10.png (the wordpress page in the wordpress website)
Saved: other_page_11.png (the email page of the microsoft application)
Saved: other_page_12.png (the youtube video player screen)
Saved: other_page_13.png (the wordpress page in the wordpress website)
Saved: other_page_14.png (the screenshoter page in the 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ All screenshots renamed clearly, duplicates removed, zipped for download!


In [ ]:
# ===============================
# Step 0: Install dependencies
# ===============================
!pip install transformers torch torchvision pillow google-colab --quiet

from PIL import Image
import os
import shutil
from google.colab import files
from zipfile import ZipFile
import hashlib
import re

from transformers import BlipProcessor, BlipForConditionalGeneration
import torch

# ===============================
# Step 1: Folders
# ===============================
input_folder = "/content/drive/MyDrive/YC/cactus_ai"  # Upload your screenshots here
output_folder = "/content/renamed_screenshots"
os.makedirs(output_folder, exist_ok=True)

# ===============================
# Step 2: Load BLIP model
# ===============================
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# ===============================
# Step 3: Helper functions
# ===============================

def clean_caption_for_filename(caption):
    """
    Converts caption to a safe, descriptive filename.
    Shortens and removes special characters.
    """
    caption = caption.lower().strip()
    caption = re.sub(r"[^a-z0-9]+", "_", caption)
    caption = re.sub(r"_+", "_", caption)
    return caption.strip("_")[:50]

def image_hash(img_path):
    img = Image.open(img_path).convert("RGB")
    return hashlib.md5(img.tobytes()).hexdigest()

# ===============================
# Step 4: Process images
# ===============================
seen_hashes = set()
name_counter = {}

for filename in sorted(os.listdir(input_folder)):
    if not filename.lower().endswith(".png"):
        continue

    old_path = os.path.join(input_folder, filename)

    # Step 4a: Skip duplicates
    img_hash = image_hash(old_path)
    if img_hash in seen_hashes:
        print(f"Skipping duplicate: {filename}")
        continue
    seen_hashes.add(img_hash)

    # Step 4b: Generate caption
    img = Image.open(old_path).convert("RGB")
    inputs = processor(img, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)

    # Step 4c: Clean caption for filename
    clean_name = clean_caption_for_filename(caption)

    # Step 4d: Add counter if needed
    if clean_name in name_counter:
        name_counter[clean_name] += 1
    else:
        name_counter[clean_name] = 1

    new_name = f"{clean_name}_{name_counter[clean_name]}.png"
    new_path = os.path.join(output_folder, new_name)

    shutil.copy(old_path, new_path)
    print(f"Saved: {new_name}  | Caption: {caption}")

# ===============================
# Step 5: Zip renamed screenshots
# ===============================
zip_name = "/content/renamed_screenshots.zip"
with ZipFile(zip_name, 'w') as zipf:
    for root, dirs, files_in in os.walk(output_folder):
        for file_in in files_in:
            zipf.write(os.path.join(root, file_in), arcname=file_in)

# ===============================
# Step 6: Download
# ===============================
files.download(zip_name)
print("✅ All screenshots renamed descriptively, duplicates removed, zipped for download!")


Saved: the_website_for_the_new_website_for_the_new_websit_1.png  | Caption: the website for the new website for the new website
Saved: the_new_version_of_the_chrome_browser_1.png  | Caption: the new version of the chrome browser
Saved: the_screen_of_the_new_google_search_page_1.png  | Caption: the screen of the new google search page
Saved: the_screen_shot_of_the_twitter_page_1.png  | Caption: the screen shot of the twitter page
Saved: the_new_version_of_the_new_version_of_the_chrome_b_1.png  | Caption: the new version of the new version of the chrome browser
Saved: the_screen_shot_of_the_new_google_search_page_1.png  | Caption: the screen shot of the new google search page
Saved: the_new_version_of_the_new_version_of_the_chrome_b_2.png  | Caption: the new version of the new version of the chrome browser
Saved: the_new_version_of_the_new_version_of_the_new_vers_1.png  | Caption: the new version of the new version of the new version of the android
Saved: the_wordpress_page_in_the_wordpr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ All screenshots renamed descriptively, duplicates removed, zipped for download!
